In [ ]:
import sys
import numpy as np
import random
import pandas as pd
import pickle
import time
import alt_min

In [ ]:
U = np.random.standard_normal((2000, 40))
V = np.random.standard_normal((40, 2000))
L = np.matmul(U, V)

In [ ]:
p = 0.1
eta = 0.1
c = 100
mask_eta = np.random.rand(L.shape[0], L.shape[1])
S = np.random.uniform(-c, c, L.shape)
S[mask_eta < 1 - eta] = 0

In [ ]:
M = L + S
mask_p = np.random.rand(M.shape[0], M.shape[1])
mask_p[mask_p < 1 - p] = 0
mask_p[mask_p != 0] = 1
M_obs = np.multiply(M, mask_p)

In [ ]:
user_col = []
item_col = []
rating_col = []
for i in range(M_obs.shape[0]):
  for j in range(M_obs.shape[1]):
    rating = M_obs[i][j]
    if rating != 0:
      user_col.append(i)
      item_col.append(j)
      rating_col.append(rating)
train = pd.DataFrame.from_dict({"user": user_col, "item": item_col, "rating": rating_col})

In [ ]:
user_col = []
item_col = []
rating_col = []
for i in range(L.shape[0]):
  for j in range(L.shape[1]):
    rating = L[i][j]
    user_col.append(i)
    item_col.append(j)
    rating_col.append(rating)
test = pd.DataFrame.from_dict({"user": user_col, "item": item_col, "rating": rating_col})

In [ ]:
model = alt_min.LeastSquares(train, test, 2000, reg=0)
model.alt_min()

In [ ]:
train.loc[0][0]

In [ ]:
train["user"]